In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Célula de Configuração do Monitoramento (adicionar no início)

# 1. Instalar bibliotecas para monitoramento
!pip install psutil pynvml

# 2. Importar tudo o que vamos precisar
import time
import threading
import psutil
import pynvml
import pandas as pd
import matplotlib.pyplot as plt

# 3. Definir a classe que fará o monitoramento
class ResourceMonitor:
    def __init__(self, interval=5):
        self.interval = interval
        self.data = []
        self._stop_event = threading.Event()
        self.thread = threading.Thread(target=self.run, daemon=True)

        # Inicializa a NVML para monitoramento da GPU
        try:
            pynvml.nvmlInit()
            self.gpu_count = pynvml.nvmlDeviceGetCount()
        except pynvml.NVMLError:
            self.gpu_count = 0
            print("AVISO: Placa NVIDIA não encontrada ou driver indisponível. O monitoramento da GPU será desativado.")

    def _get_gpu_ram_usage(self):
        if self.gpu_count == 0:
            return 0
        handle = pynvml.nvmlDeviceGetHandleByIndex(0)
        info = pynvml.nvmlDeviceGetMemoryInfo(handle)
        return info.used / (1024**3)  # Convertido para GB

    def run(self):
        """O método que roda em segundo plano para coletar dados."""
        start_time = time.time()
        while not self._stop_event.is_set():
            timestamp = time.time() - start_time

            # Coleta de dados
            sys_ram_used = psutil.virtual_memory().used / (1024**3) # GB
            gpu_ram_used = self._get_gpu_ram_usage() # GB
            disk_used = psutil.disk_usage('/').used / (1024**3) # GB

            self.data.append([timestamp, sys_ram_used, gpu_ram_used, disk_used])
            time.sleep(self.interval)

        if self.gpu_count > 0:
            pynvml.nvmlShutdown()

    def start(self):
        """Inicia o monitoramento."""
        print("Iniciando monitoramento de recursos...")
        self.thread.start()

    def stop(self):
        """Para o monitoramento."""
        self._stop_event.set()
        self.thread.join()
        print("Monitoramento de recursos finalizado.")
        return pd.DataFrame(self.data, columns=['Tempo (s)', 'RAM Sistema (GB)', 'RAM GPU (GB)', 'Disco (GB)'])

    def plot(self):
        """Plota os dados coletados."""
        df = self.stop()

        if df.empty:
            print("Nenhum dado de monitoramento foi coletado.")
            return

        fig, axes = plt.subplots(3, 1, figsize=(12, 15), sharex=True)
        fig.suptitle('Utilização de Recursos do Sistema Durante a Execução', fontsize=16)

        # Gráfico de RAM do Sistema
        axes[0].plot(df['Tempo (s)'], df['RAM Sistema (GB)'], label='RAM do Sistema Utilizada', color='blue')
        axes[0].set_ylabel('Uso (GB)')
        axes[0].set_title('Uso de RAM do Sistema')
        axes[0].grid(True)
        axes[0].legend()
        axes[0].fill_between(df['Tempo (s)'], df['RAM Sistema (GB)'], alpha=0.1, color='blue')

        # Gráfico de RAM da GPU
        if self.gpu_count > 0:
            axes[1].plot(df['Tempo (s)'], df['RAM GPU (GB)'], label='RAM da GPU Utilizada', color='green')
        else:
            axes[1].text(0.5, 0.5, 'Monitoramento de GPU não disponível', ha='center', va='center')
        axes[1].set_ylabel('Uso (GB)')
        axes[1].set_title('Uso de RAM da GPU')
        axes[1].grid(True)
        axes[1].legend()
        axes[1].fill_between(df['Tempo (s)'], df['RAM GPU (GB)'], alpha=0.1, color='green')

        # Gráfico de Uso de Disco
        axes[2].plot(df['Tempo (s)'], df['Disco (GB)'], label='Espaço em Disco Utilizado', color='red')
        axes[2].set_xlabel('Tempo (segundos)')
        axes[2].set_ylabel('Uso (GB)')
        axes[2].set_title('Uso de Disco')
        axes[2].grid(True)
        axes[2].legend()
        axes[2].fill_between(df['Tempo (s)'], df['Disco (GB)'], alpha=0.1, color='red')

        plt.tight_layout(rect=[0, 0, 1, 0.96])
        plt.show()

# 4. Iniciar o cronômetro e o monitoramento
# (Coloque estas 2 linhas logo antes do seu código principal começar a rodar)
tempo_inicial = time.time()
monitor = ResourceMonitor(interval=5) # O 'interval' é em segundos
monitor.start()

In [ ]:
!pip install midi2audio

# Imports
import os
import numpy as np
import pandas as pd
from collections import Counter
import random
import warnings
from music21 import converter, instrument, note, chord, stream
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adamax
import tensorflow.keras.utils
from midi2audio import FluidSynth
from google.colab import drive
from tqdm import tqdm # Importação da tqdm

# Configurações
warnings.filterwarnings("ignore")
np.random.seed(42)

# Caminhos
dataset_path = "/content/drive/MyDrive/test-code/classical-music-midi/chopin" # Caminho para a pasta do MAESTRO
soundfont_path = "/content/drive/MyDrive/soundfonts"
output_midi = "Melody_Generated.mid"
output_wav = "Melody_Generated.wav"

In [ ]:
# Carregar MIDIs com barra de progresso e tratamento de erros
all_midis = []
print(f"Carregando arquivos MIDI de: {dataset_path}")

# Usamos a tqdm para criar uma barra de progresso
file_list = os.listdir(dataset_path)
for i in tqdm(file_list, desc="Processando arquivos"):
    # Usamos .lower() para garantir que a extensão seja lida corretamente
    if i.lower().endswith(('.mid', '.midi')):
        try:
            tr = os.path.join(dataset_path, i)
            midi = converter.parse(tr)
            all_midis.append(midi)
        except Exception as e:
            # Adicionado um try-except para pular arquivos corrompidos
            print(f"\nAVISO: Não foi possível processar o arquivo {i}. Erro: {e}")
            continue

print(f"\nCarregamento concluído. {len(all_midis)} arquivos foram carregados com sucesso.")

In [ ]:
# Extrair notas
def extract_notes(file):
    notes = []
    for j in file:
        songs = instrument.partitionByInstrument(j)
        for part in songs.parts:
            pick = part.recurse()
            for element in pick:
                if isinstance(element, note.Note):
                    notes.append(str(element.pitch))
                elif isinstance(element, chord.Chord):
                    notes.append(".".join(str(n) for n in element.normalOrder))
    return notes

Corpus = extract_notes(all_midis)
print("Total notes in all the Chopin midis:", len(Corpus))

# Remover notas raras
count_num = Counter(Corpus)
rare_note = [k for k, v in count_num.items() if v < 100]
Corpus = [element for element in Corpus if element not in rare_note]

# Mapeamento
symb = sorted(list(set(Corpus)))
mapping = dict((c, i) for i, c in enumerate(symb))
reverse_mapping = dict((i, c) for i, c in enumerate(symb))

In [ ]:
# Preparar dados
length = 40
features = []
targets = []

for i in range(0, len(Corpus) - length, 1):
    feature = Corpus[i:i + length]
    target = Corpus[i + length]
    features.append([mapping[j] for j in feature])
    targets.append(mapping[target])

X = np.reshape(features, (len(targets), length, 1)) / float(len(symb))
y = tensorflow.keras.utils.to_categorical(targets)

X_train, X_seed, y_train, y_seed = train_test_split(X, y, test_size=0.2, random_state=42)

# Modelo
# OPÇÃO C: MODELO MAIS PROFUNDO E MAIS LARGO
model = Sequential()
# Camada 1 (Larga)
model.add(LSTM(1024, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2)) # Aumentei um pouco o dropout para combater o overfitting
# Camada 2 (Intermediária)
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.2))
# Camada 3 (Final Recorrente)
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
# Treinar
history = model.fit(X_train, y_train, batch_size=256, epochs=100)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# --- Dados extraídos do seu log de treinamento ---
epochs = list(range(1, 101))
loss_values = [
    4.5321, 4.3844, 4.3637, 4.3447, 4.3156, 4.2838, 4.2153, 4.1489, 4.0622, 3.9208,
    3.7515, 3.5441, 3.3402, 3.1347, 2.8859, 2.6849, 2.4802, 2.2499, 2.0460, 1.8288,
    1.6387, 1.4788, 1.3251, 1.1693, 1.0244, 0.9005, 0.8016, 0.7013, 0.6290, 0.5548,
    0.5042, 0.4496, 0.4167, 0.3713, 0.3452, 0.3204, 0.3016, 0.2761, 0.2571, 0.2603,
    0.2499, 0.2303, 0.2277, 0.2317, 0.2275, 0.2194, 0.2166, 0.2001, 0.2018, 0.1942,
    0.1834, 0.1774, 0.1722, 0.1753, 0.1696, 0.1854, 0.1716, 0.1720, 0.1740, 0.1637,
    0.1609, 0.1593, 0.1569, 0.1404, 0.1454, 0.1521, 0.1446, 0.1390, 0.1471, 0.1456,
    0.1451, 0.1461, 0.1281, 0.1145, 0.1277, 0.1327, 0.1413, 0.1258, 0.1271, 0.1327,
    0.1436, 0.1209, 0.1191, 0.1188, 0.1168, 0.1246, 0.1258, 0.1333, 0.1246, 0.1200,
    0.1151, 0.1194, 0.1090, 0.1026, 0.1166, 0.1107, 0.1064, 0.1117, 0.1107, 0.1038
]
accuracy_values = [
    0.0283, 0.0371, 0.0410, 0.0430, 0.0444, 0.0466, 0.0564, 0.0625, 0.0777, 0.0972,
    0.1249, 0.1599, 0.1936, 0.2342, 0.2787, 0.3161, 0.3577, 0.4102, 0.4534, 0.5087,
    0.5480, 0.5897, 0.6257, 0.6670, 0.7050, 0.7383, 0.7616, 0.7925, 0.8112, 0.8361,
    0.8479, 0.8617, 0.8727, 0.8871, 0.8957, 0.9049, 0.9083, 0.9154, 0.9242, 0.9212,
    0.9260, 0.9301, 0.9325, 0.9293, 0.9296, 0.9339, 0.9342, 0.9386, 0.9370, 0.9414,
    0.9447, 0.9478, 0.9474, 0.9477, 0.9474, 0.9445, 0.9486, 0.9475, 0.9479, 0.9512,
    0.9518, 0.9502, 0.9518, 0.9573, 0.9556, 0.9535, 0.9557, 0.9570, 0.9565, 0.9552,
    0.9568, 0.9557, 0.9610, 0.9663, 0.9607, 0.9587, 0.9560, 0.9631, 0.9616, 0.9587,
    0.9560, 0.9640, 0.9652, 0.9649, 0.9650, 0.9653, 0.9614, 0.9593, 0.9623, 0.9640,
    0.9652, 0.9638, 0.9681, 0.9696, 0.9669, 0.9672, 0.9674, 0.9657, 0.9678, 0.9677
]
# ----------------------------------------------------

# --- Criar a Figura com dois subplots (um em cima do outro) ---
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10), sharex=True)

# --- Gráfico 1: Perda (Loss) ---
ax1.plot(epochs, loss_values, 'r.-', label='Perda de Treinamento')
ax1.set_title('Curva de Perda (Loss) por Epoch', fontsize=16)
ax1.set_ylabel('Valor da Perda (Loss)', fontsize=12)
ax1.grid(True, linestyle='--', alpha=0.6)
ax1.legend()
# Definir marcadores do eixo X para ambos os gráficos
ax1.set_xticks(np.arange(0, 101, 10)) # Marcadores a cada 10 épocas

# --- Gráfico 2: Acurácia (Accuracy) ---
ax2.plot(epochs, accuracy_values, 'g.-', label='Acurácia de Treinamento')
ax2.set_title('Curva de Acurácia (Accuracy) por Epoch', fontsize=16)
ax2.set_xlabel('Epoch', fontsize=12)
ax2.set_ylabel('Acurácia', fontsize=12)
ax2.grid(True, linestyle='--', alpha=0.6)
ax2.legend()

# Ajustar o layout e exibir
plt.tight_layout()
plt.show()

In [ ]:
# Gerar música
def chords_n_notes(Snippet):
    Melody = []
    offset = 0
    for i in Snippet:
        if ("." in i or i.isdigit()):
            chord_notes = i.split(".")
            notes = [note.Note(int(j)) for j in chord_notes]
            chord_snip = chord.Chord(notes)
            chord_snip.offset = offset
            Melody.append(chord_snip)
        else:
            note_snip = note.Note(i)
            note_snip.offset = offset
            Melody.append(note_snip)
        offset += 1
    return stream.Stream(Melody)

def Malody_Generator(Note_Count):
    seed = X_seed[np.random.randint(0, len(X_seed)-1)]
    Notes_Generated = []
    for _ in range(Note_Count):
        seed = seed.reshape(1, length, 1)
        prediction = model.predict(seed, verbose=0)[0]
        prediction = np.log(prediction + 1e-8)
        exp_preds = np.exp(prediction)
        prediction = exp_preds / np.sum(exp_preds)
        index = np.argmax(prediction)
        Notes_Generated.append(index)
        seed = np.insert(seed[0], len(seed[0]), index / float(len(symb)))
        seed = seed[1:]
    Music = [reverse_mapping[char] for char in Notes_Generated]
    Melody = chords_n_notes(Music)
    return Music, Melody

Music_notes, Melody = Malody_Generator(100)
Melody.write('midi', output_midi)


In [ ]:
!apt-get update
!apt-get install fluidsynth

In [ ]:
# Converter .mid para .wav
fs = FluidSynth(soundfont_path)
fs.midi_to_audio(output_midi, output_wav)
print(f"Arquivo .wav gerado: {output_wav}")

# Reproduzir no Colab
import IPython.display as ipd
ipd.Audio(output_wav)


In [ ]:
# Célula de Finalização e Plotagem (adicionar no final)

# 1. Parar o cronômetro e calcular o tempo total
tempo_final = time.time()
tempo_total_segundos = tempo_final - tempo_inicial

# Formatando o tempo para horas, minutos e segundos
horas = int(tempo_total_segundos // 3600)
minutos = int((tempo_total_segundos % 3600) // 60)
segundos = int(tempo_total_segundos % 60)

print("\n--- Relatório Final de Execução ---")
print(f"Tempo de Execução Total: {horas}h {minutos}min {segundos}s")

# 2. Parar o monitor e gerar o gráfico de uso de recursos
# A função plot() já chama o stop() e processa os dados
monitor.plot()